In [2]:
#pip install --upgrade selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd

url_home = 'https://s.cafef.vn/'

def scrape_article_details(link):
    article_response = requests.get(link)
    if article_response.status_code == 200:
        article_soup = BeautifulSoup(article_response.text, "html.parser")
        excerpts = article_soup.find("h2", {'class': "intro"})
        return excerpts.text.strip() if excerpts else None
    return None

def scrape_page_selenium(stock_code_input):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run Chrome in headless mode (no GUI)
    
    with webdriver.Chrome(options=options) as driver:
        url_template = f"https://s.cafef.vn/tin-doanh-nghiep/{stock_code_input}/event.chn"
        driver.get(url_template)
        news_data = []
        while True:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "divTopEvents")))
            soup = BeautifulSoup(driver.page_source, "html.parser")
            container = soup.find("div", {"id": "divTopEvents"})
            articles_list = container.find_all("li")
            if not articles_list:
                break
            for article in articles_list:
                title = article.find('a', class_='docnhanhTitle').text.strip()
                time = article.find('span', class_='timeTitle').text.strip()
                link = article.find('a', class_='docnhanhTitle')['href']
                full_link = url_home + link
                excerpts = scrape_article_details(full_link)

                news_data.append({
                    'Tiêu đề': title,
                    'Thời gian': time,
                    'Link': full_link,
                    'Nội dung chính': excerpts
                })

            try:
                next_button = driver.find_element(By.ID, "spanNext")
                if "disabled" in next_button.get_attribute("class"):
                    break
                else:
                    next_button.click()
            except:
                break

    df = pd.DataFrame(news_data)
    df.to_excel(f'Data_{stock_code_input}.xlsx', index=False)

# Nhập mã chứng khoán từ người dùng
stock_code_input = input("Nhập mã chứng khoán: ").upper()
scrape_page_selenium(stock_code_input)

Nhập mã chứng khoán:  CTR
